# 量子神经网络在自然语言处理中的应用


`Linux` `CPU` `全流程` `初级` `中级` `高级`

[![](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/training/source_zh_cn/advanced_use/qnn_for_nlp.ipynb)

## 概述

在自然语言处理过程中，词嵌入（Word embedding）是其中的重要步骤，它是一个将高维度空间的词向量嵌入到一个维数更低的连续向量空间的过程。当给予神经网络的语料信息不断增加时，网络的训练过程将越来越困难。利用量子力学的态叠加和纠缠等特性，我们可以利用量子神经网络来处理这些经典语料信息，加入其训练过程，并提高收敛精度。下面，我们将简单地搭建一个量子经典混合神经网络来完成一个词嵌入任务。


## 环境准备

导入本教程所依赖模块


In [1]:
import numpy as np
import time
from projectq.ops import QubitOperator
import mindspore.ops as ops
import mindspore.dataset as ds
from mindspore import nn
from mindspore.train.callback import LossMonitor
from mindspore import Model
from mindquantum.nn import MindQuantumLayer
from mindquantum import Hamiltonian, Circuit, RX, RY, X, H, UN

本教程实现的是一个[CBOW模型](https://blog.csdn.net/u010665216/article/details/78724856)，即利用某个词所处的环境来预测该词。例如对于“I love natural language processing”这句话，我们可以将其切分为5个词，\["I", "love", "natural", "language", "processing”\]，在所选窗口为2时，我们要处理的问题是利用\["I", "love", "language", "processing"\]来预测出目标词汇"natural"。这里我们以窗口为2为例，搭建如下的量子神经网络，来完成词嵌入任务。

![quantum word embedding](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/advanced_use/images/qcbow.png)


这里，编码线路会将"I"、"love"、"language"和"processing"的编码信息编码到量子线路中，待训练的量子线路由四个Ansatz线路构成，最后我们在量子线路末端对量子比特做$\text{Z}$基矢上的测量，具体所需测量的比特的个数由所需嵌入空间的维数确定。

## 数据预处理

我们对所需要处理的语句进行处理，生成关于该句子的词典，并根据窗口大小来生成样本点。


In [2]:
def GenerateWordDictAndSample(corpus, window=2):
    all_words = corpus.split()
    word_set = list(set(all_words))
    word_set.sort()
    word_dict = {w: i for i,w in enumerate(word_set)}
    sampling = []
    for index, word in enumerate(all_words[window:-window]):
        around = []
        for i in range(index, index + 2*window + 1):
            if i != index + window:
                around.append(all_words[i])
        sampling.append([around,all_words[index + window]])
    return word_dict, sampling

In [3]:
word_dict, sample = GenerateWordDictAndSample("I love natural language processing")
print(word_dict)
print('word dict size: ', len(word_dict))
print('samples: ', sample)
print('number of samples: ', len(sample))

{'I': 0, 'language': 1, 'love': 2, 'natural': 3, 'processing': 4}
word dict size:  5
samples:  [[['I', 'love', 'language', 'processing'], 'natural']]
number of samples:  1


根据如上信息，我们得到该句子的词典大小为5，能够产生一个样本点。

## 编码线路

为了简单起见，我们使用的编码线路由$\text{RX}$旋转门构成，结构如下。

![encoder circuit](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/advanced_use/images/encoder.png)

我们对每个量子门都作用一个$\text{RX}$旋转门。

In [4]:
def GenerateEncoderCircuit(n_qubits, prefix=''):
    if len(prefix) != 0 and prefix[-1] != '_':
        prefix += '_'
    circ = Circuit()
    for i in range(n_qubits):
        circ += RX(prefix + str(i)).on(i)
    return circ

In [5]:
GenerateEncoderCircuit(3,prefix='e')

RX(e_0|0)
RX(e_1|1)
RX(e_2|2)

我们通常用$\left|0\right>$和$\left|1\right>$来标记二能级量子比特的两个状态，由态叠加原理，量子比特还可以处于这两个状态的叠加态：

$$\left|\psi\right>=\alpha\left|0\right>+\beta\left|1\right>$$

对于$n$比特的量子态，其将处于$2^n$维的希尔伯特空间中。对于上面由5个词构成的词典，我们只需要$\lceil \log_2 5 \rceil=3$个量子比特即可完成编码，这也体现出量子计算的优越性。

例如对于上面词典中的"love"，其对应的标签为2，2的二进制表示为`010`，我们只需将编码线路中的`e_0`、`e_1`和`e_2`分别设为$0$、$\pi$和$0$即可。下面我们通过`Evolution`算子来验证以下。

In [6]:
from mindquantum.nn import generate_evolution_operator
from mindspore import context
from mindspore import Tensor

n_qubits = 3 # number of qubits of this quantum circuit
label = 2 # label need to encode
label_bin = bin(label)[-1:1:-1].ljust(n_qubits,'0') # binary form of label
label_array = np.array([int(i)*np.pi for i in label_bin]).astype(np.float32) # parameter value of encoder
encoder = GenerateEncoderCircuit(n_qubits, prefix='e') # encoder circuit
encoder_para_names = encoder.parameter_resolver().para_name # parameter names of encoder

print("Label is: ", label)
print("Binary label is: ", label_bin)
print("Parameters of encoder is: \n", np.round(label_array, 5))
print("Encoder circuit is: \n", encoder)
print("Encoder parameter names are: \n", encoder_para_names)

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")
# quantum state evolution operator
evol = generate_evolution_operator(param_names=encoder_para_names, circuit=encoder)
state = evol(Tensor(label_array))
state = state.asnumpy()
quantum_state = state[:, 0] + 1j * state[:, 1]
amp = np.round(np.abs(quantum_state)**2, 3)

print("Amplitude of quantum state is: \n", amp)
print("Label in quantum state is: ", np.argmax(amp))

Label is:  2
Binary label is:  010
Parameters of encoder is: 
 [0.      3.14159 0.     ]
Encoder circuit is: 
 RX(e_0|0)
RX(e_1|1)
RX(e_2|2)
Encoder parameter names are: 
 ['e_0', 'e_1', 'e_2']
Amplitude of quantum state is: 
 [0. 0. 1. 0. 0. 0. 0. 0.]
Label in quantum state is:  2


通过上面的验证，我们发现，对于标签为2的数据，最后得到量子态的振幅最大的位置也是2，因此得到的量子态正是对输入标签的编码。我们将对数据编码生成参数数值的过程总结成如下函数。

In [7]:
def GenerateTrainData(sample, word_dict):
    n_qubits = np.int(np.ceil(np.log2(1 + max(word_dict.values()))))
    data_x = []
    data_y = []
    for around, center in sample:
        data_x.append([])
        for word in around:
            label = word_dict[word]
            label_bin = bin(label)[-1:1:-1].ljust(n_qubits,'0')
            label_array = [int(i)*np.pi for i in label_bin]
            data_x[-1].extend(label_array)
        data_y.append(word_dict[center])
    return np.array(data_x).astype(np.float32), np.array(data_y).astype(np.int32)

In [8]:
GenerateTrainData(sample, word_dict)

(array([[0.       , 0.       , 0.       , 0.       , 3.1415927, 0.       ,
         3.1415927, 0.       , 0.       , 0.       , 0.       , 3.1415927]],
       dtype=float32),
 array([3], dtype=int32))

根据上面的结果，我们将4个输入的词编码的信息合并为一个更长向量，便于后续神经网络调用。

## Ansatz线路

Ansatz线路的选择多种多样，我们选择如下的量子线路作为Ansatz线路，它的一个单元由一层$\text{RY}$门和一层$\text{CNOT}$门构成，对此单元重复$p$次构成整个Ansatz线路。

![ansatz circuit](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/advanced_use/images/ansatz.png)

定义如下函数生成Ansatz线路。

In [9]:
def GenerateAnsatzCircuit(n_qubits, layers, prefix=''):
    if len(prefix) != 0 and prefix[-1] != '_':
        prefix += '_'
    circ = Circuit()
    for l in range(layers):
        for i in range(n_qubits):
            circ += RY(prefix + str(l) + '_' + str(i)).on(i)
        for i in range(l % 2, n_qubits, 2):
            if i < n_qubits and i + 1 < n_qubits:
                circ += X.on(i + 1, i)
    return circ

In [10]:
GenerateAnsatzCircuit(5, 2, 'a')

RY(a_0_0|0)
RY(a_0_1|1)
RY(a_0_2|2)
RY(a_0_3|3)
RY(a_0_4|4)
X(1 <-: 0)
X(3 <-: 2)
RY(a_1_0|0)
RY(a_1_1|1)
RY(a_1_2|2)
RY(a_1_3|3)
RY(a_1_4|4)
X(2 <-: 1)
X(4 <-: 3)

## 测量

我们把对不同比特位上的测量结果作为降维后的数据。具体过程与比特编码类似，例如当我们想将词向量降维为5维向量时，对于第3维的数据可以如下产生：

- 3对应的二进制为`00011`。
- 测量量子线路末态对$Z_0Z_1$哈密顿量的期望值。

下面函数将给出产生各个维度上数据所需的哈密顿量（hams），其中`n_qubits`表示线路的比特数，`dims`表示词嵌入的维度：

In [11]:
def GenerateEmbeddingHamiltonian(dims, n_qubits):
    hams = []
    for i in range(dims):
        s = ''
        for j, k in enumerate(bin(i + 1)[-1:1:-1]):
            if k == '1':
                s = s + 'Z' + str(j) + ' '
        hams.append(Hamiltonian(QubitOperator(s)))
    return hams

In [12]:
GenerateEmbeddingHamiltonian(5, 5)

[1.0 Z0, 1.0 Z1, 1.0 Z0 Z1, 1.0 Z2, 1.0 Z0 Z2]

## 量子版词向量嵌入层

量子版词向量嵌入层结合前面的编码量子线路和待训练量子线路，以及测量哈密顿量，将`num_embedding`个词嵌入为`embedding_dim`维的词向量。这里我们还在量子线路的最开始加上了Hadamard门，将初态制备为均匀叠加态，用以提高量子神经网络的表达能力。

下面，我们定义量子嵌入层，它将返回一个量子线路模拟算子。

In [13]:
def QEmbedding(num_embedding, embedding_dim, window, layers, n_threads):
    n_qubits = int(np.ceil(np.log2(num_embedding)))
    hams = GenerateEmbeddingHamiltonian(embedding_dim, n_qubits)
    circ = Circuit()
    circ = UN(H, n_qubits)
    encoder_param_name = []
    ansatz_param_name = []
    for w in range(2 * window):
        encoder = GenerateEncoderCircuit(n_qubits, 'Encoder_' + str(w))
        ansatz = GenerateAnsatzCircuit(n_qubits, layers, 'Ansatz_' + str(w))
        encoder.no_grad()
        circ += encoder
        circ += ansatz
        encoder_param_name.extend(list(encoder.parameter_resolver()))
        ansatz_param_name.extend(list(ansatz.parameter_resolver()))
    net = MindQuantumLayer(encoder_param_name,
                           ansatz_param_name,
                           circ,
                           hams,
                           n_threads=n_threads)
    return net

整个训练模型跟经典网络类似，由一个嵌入层和两个全连通层构成，然而此处的嵌入层是由量子神经网络构成。下面定义量子神经网络CBOW。

In [14]:
class CBOW(nn.Cell):
    def __init__(self, num_embedding, embedding_dim, window, layers, n_threads,
                 hidden_dim):
        super(CBOW, self).__init__()
        self.embedding = QEmbedding(num_embedding, embedding_dim, window,
                                    layers, n_threads)
        self.dense1 = nn.Dense(embedding_dim, hidden_dim)
        self.dense2 = nn.Dense(hidden_dim, num_embedding)
        self.relu = ops.ReLU()

    def construct(self, x):
        embed = self.embedding(x)
        out = self.dense1(embed)
        out = self.relu(out)
        out = self.dense2(out)
        return out

下面我们对一个稍长的句子来进行训练。首先定义`LossMonitorWithCollection`用于监督收敛过程，并搜集收敛过程的损失。

In [15]:
class LossMonitorWithCollection(LossMonitor):
    def __init__(self, per_print_times=1):
        super(LossMonitorWithCollection, self).__init__(per_print_times)
        self.loss = []
        
    def begin(self, run_context):
        self.begin_time = time.time()
        
    def end(self, run_context):
        self.end_time = time.time()
        print('Total time used: {}'.format(self.end_time - self.begin_time))
        
    def epoch_begin(self, run_context):
        self.epoch_begin_time = time.time()
        
    def epoch_end(self, run_context):
        cb_params = run_context.original_args()
        self.epoch_end_time = time.time()
        if self._per_print_times != 0 and cb_params.cur_step_num % self._per_print_times == 0:
            print('')
    
    def step_end(self, run_context):
        cb_params = run_context.original_args()
        loss = cb_params.net_outputs

        if isinstance(loss, (tuple, list)):
            if isinstance(loss[0], Tensor) and isinstance(loss[0].asnumpy(), np.ndarray):
                loss = loss[0]

        if isinstance(loss, Tensor) and isinstance(loss.asnumpy(), np.ndarray):
            loss = np.mean(loss.asnumpy())

        cur_step_in_epoch = (cb_params.cur_step_num - 1) % cb_params.batch_num + 1

        if isinstance(loss, float) and (np.isnan(loss) or np.isinf(loss)):
            raise ValueError("epoch: {} step: {}. Invalid loss, terminating training.".format(
                cb_params.cur_epoch_num, cur_step_in_epoch))
        self.loss.append(loss)
        if self._per_print_times != 0 and cb_params.cur_step_num % self._per_print_times == 0:
            print("\repoch: %+3s step: %+3s time: %5.5s, loss is %5.5s" % (cb_params.cur_epoch_num, cur_step_in_epoch, time.time() - self.epoch_begin_time, loss), flush=True, end='')


接下来，利用量子版本的`CBOW`来对一个长句进行词嵌入。运行之前请在终端运行`export OMP_NUM_THREADS=4`，将量子模拟器的线程数设置为4个，当所需模拟的量子系统比特数较多时，可设置更多的线程数来提高模拟效率。

In [16]:
import mindspore as ms
from mindspore import context
from mindspore import Tensor
context.set_context(mode=context.GRAPH_MODE, device_target="CPU")
corpus = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

ms.set_seed(42)
window_size = 2
embeding_dim = 10
hidden_dim = 128
word_dict, sample = GenerateWordDictAndSample(corpus, window=window_size)
train_x,train_y = GenerateTrainData(sample, word_dict)

train_loader = ds.NumpySlicesDataset({
    "around": train_x,
    "center": train_y
},shuffle=False).batch(3)
net = CBOW(len(word_dict), embeding_dim, window_size, 3, 4, hidden_dim)
net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
net_opt = nn.Momentum(net.trainable_params(), 0.01, 0.9)
loss_monitor = LossMonitorWithCollection(500)
model = Model(net, net_loss, net_opt)
model.train(350, train_loader, callbacks=[loss_monitor])

epoch:  25 step:  20 time: 0.580, loss is 3.154
epoch:  50 step:  20 time: 0.532, loss is 2.945
epoch:  75 step:  20 time: 0.643, loss is 0.226
epoch: 100 step:  20 time: 0.562, loss is 0.016
epoch: 125 step:  20 time: 0.516, loss is 0.002
epoch: 150 step:  20 time: 0.658, loss is 0.004
epoch: 175 step:  20 time: 0.554, loss is 0.000
epoch: 200 step:  20 time: 0.504, loss is 5.146
epoch: 225 step:  20 time: 0.493, loss is 0.001
epoch: 250 step:  20 time: 0.549, loss is 0.091
epoch: 275 step:  20 time: 0.502, loss is 0.018
epoch: 300 step:  20 time: 0.616, loss is 0.000
epoch: 325 step:  20 time: 0.545, loss is 0.004
epoch: 350 step:  20 time: 0.499, loss is 0.003
Total time used: 211.59854197502136


打印收敛过程中的损失函数值：

In [ ]:
import matplotlib.pyplot as plt

plt.plot(loss_monitor.loss,'.')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.show()

得到收敛图为

![nlp loss](https://gitee.com/mindspore/docs/raw/master/tutorials/training/source_zh_cn/advanced_use/images/nlp_loss.png)

通过如下方法打印量子嵌入层的量子线路中的参数：

In [17]:
net.embedding.weight.asnumpy()

array([ 2.2985651e+00, -5.9529102e-01, -1.4730859e+00, -8.7775314e-01,
        1.5892730e+00,  1.2431808e+00,  4.6158323e-01,  1.4650096e-02,
       -1.5554352e+00,  1.7425102e+00,  8.0435050e-01,  2.9033291e-01,
        4.5533547e-01, -3.3123956e+00, -5.0230217e-03,  1.4551424e+00,
       -2.2800403e+00,  2.8021520e-01, -6.8450904e-01,  3.2497895e+00,
       -6.8959081e-01, -1.9466689e+00, -9.2352051e-01,  1.0398046e+00,
       -8.0794984e-01,  9.3039703e-01,  1.2520922e+00,  7.7605700e-01,
       -2.2833204e-01, -6.5232110e-01, -8.0807763e-01,  3.1698236e+00,
       -3.1225615e+00,  1.6280267e+00, -2.9284138e-02, -6.4399004e-01,
       -8.4129974e-02,  1.6495787e-02,  6.7466281e-02,  4.9414963e-01,
       -9.5305812e-01,  9.5651263e-01, -2.4311234e-01, -2.3619778e-01,
        1.6919808e+00,  1.3713958e+00, -1.8677666e+00,  2.2548558e-01,
       -2.4450117e-01,  1.6238995e-01, -2.4916198e-02, -4.4683918e-01,
        1.2642978e+00,  2.4357845e-01, -7.0971340e-02,  1.9800837e-01,
      

## 参考文献

[1] Tomas Mikolov, Kai Chen, Greg Corrado, Jeffrey Dean. [Efficient Estimation of Word Representations in
Vector Space](https://arxiv.org/pdf/1301.3781.pdf)